In [1]:
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle

In [2]:
green = plt.imread('images/green.png')
yellow = plt.imread('images/yellow.png')
red = plt.imread('images/red.png')
white_space = np.ones(shape=(8, 4, 3))

In [3]:
def get_odds(times):
    labels = [-100] * 5 # bad
    labels.extend([0] * 50)
    labels.extend([1] * 45) # good
    
    success = 0

    for i in range(times):
        shuffle(labels)
        if sum(labels[:10]) > 7:
            success += 1

    return success/times*100

#print(get_odds(100000))

In [4]:
def choose_image(label):
    if label > 8:
        return green
    elif label <= 4:
        return red
    else:
        return yellow

def get_image(labels):
    image = choose_image(labels[0])

    for i in range(1, len(labels)):
        image = np.hstack((image, choose_image(labels[i])))
        
        if i == 89:
            image = np.hstack((image, white_space))

    return image

In [5]:
def get_test():
    my_df = pd.read_csv('data/my_ratings.csv', encoding='latin-1')
    my_df = my_df[my_df['Title Type'] == 'movie']
    my_df = my_df.sample(frac=1).reset_index(drop=True)

    good_idx = my_df[my_df['Your Rating'] > 8].index.values
    normal_idx = my_df[(my_df['Your Rating'] <= 8) & (my_df['Your Rating'] > 4)].index.values
    bad_idx = my_df[my_df['Your Rating'] <= 4].index.values

    test_idx = np.append(good_idx[:45], normal_idx[:50])
    test_idx = np.append(test_idx, bad_idx[:5])

    #test_df = my_df.iloc[test_idx].sort_values('Your Rating') # Ideal
    #test_df = my_df.iloc[test_idx].sample(frac=1) # Random
    test_df = my_df.iloc[test_idx].sort_values('IMDb Rating')
    
    test_df = test_df.reset_index(drop=True)
    #print(test_df[['Title', 'IMDb Rating', 'Your Rating']])
    return test_df['Your Rating'].values

#plt.imsave('images/list_.png', get_image(get_test()))

In [ ]:
def df_to_markdown(df):
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    print(df_formatted.to_csv(sep="|", index=False))